# Trinkgeld Calculator

In [142]:
import pandas as pd
from datetime import datetime

path = ('./data/sample01.xlsx')
xl = pd.ExcelFile(path)

employees = xl.sheet_names
employees.remove('Übersicht')
print(employees)

['Emelie Appl', 'Açelya Basili', 'Fabrizio Bellia', 'Anna Sophie Blessing', 'Sabulla Bullach', 'Manuel Bär', 'Markus Fuchs', 'Anna Graf', 'Ekaterina Grashchenko', 'Laura Greidenweiß', 'Svenja Halamek', 'Marie Jaussi', 'Okechulowu Ohaebsim', 'Philine Ostermayer', 'Damiano Parziale', 'Luca Principi co Bischoff', 'Pia Schmolly', 'Mauro Sirigu', 'Matthias Strobel', 'Luca Tegeder', 'Jana Wegenke', 'Lilli Wittig', 'Jennifer Wöhrlin']


In [143]:
class WorkDay:
    def __init__(self, day, hours):
        self.day = datetime.strptime(day[3:], "%d.%m.%Y")
        self.hours = self._parse_hours(hours)
    
    def add(self, workday):
        if (self.day != workday.day):
            raise ValueError('Days of workdays must match to sum up work hours!')
        else:
            self.hours += workday.hours
    
    def _parse_hours(self, hours):
        hours, minutes = map(int, hours.split(':'))
        return round(hours + (minutes / 60), 2)      
    
    def __repr__(self):
        return f"{self.day}: {self.hours}"
    
class Employee:
    def __init__(self, name, work_days):
        self.name = name
        self.work_days = work_days

    def __repr__(self):
        work_days = '\n'.join('\t' + str(work_day) for work_day in self.work_days)
        return f"{self.name}: \n{work_days}"

def parse_hours_for(name):
    sheet = xl.parse(name, header=None)
    header = sheet[sheet[0] == 'Tag'].index[0]

    data = xl.parse(name, header=header)
    data = data[['Tag', 'Startzeit', 'Endzeit', 'Pause (min)', 'Dauer netto (h)']]
    data = data[data['Startzeit'].notnull()]
    
    work_days = {}
    for _, row in data.iterrows():
        work_day = WorkDay(day=row['Tag'], hours=row['Dauer netto (h)'])
        if work_day.day in work_days:
            work_days[work_day.day].add(work_day)
        else:
            work_days[work_day.day] = work_day

    employee = Employee(name=name, work_days=work_days.values())

    return employee


In [144]:
employees_work_hours = [parse_hours_for(name) for name in employees]

for employee in employees_work_hours:
    print(employee)

Emelie Appl: 
	2025-03-11 00:00:00: 5.05
	2025-03-12 00:00:00: 5.93
Açelya Basili: 
	2025-03-10 00:00:00: 4.52
Fabrizio Bellia: 
	2025-03-12 00:00:00: 11.0
	2025-03-13 00:00:00: 11.65
	2025-03-14 00:00:00: 6.07
	2025-03-15 00:00:00: 5.0
	2025-03-16 00:00:00: 6.82
Anna Sophie Blessing: 
	2025-03-10 00:00:00: 6.0
	2025-03-12 00:00:00: 5.82
	2025-03-13 00:00:00: 4.55
Sabulla Bullach: 
	2025-03-12 00:00:00: 7.13
	2025-03-14 00:00:00: 6.97
	2025-03-15 00:00:00: 7.43
	2025-03-16 00:00:00: 7.87
Manuel Bär: 
	2025-03-11 00:00:00: 7.98
	2025-03-13 00:00:00: 6.83
	2025-03-14 00:00:00: 1.85
Markus Fuchs: 
	2025-03-11 00:00:00: 6.12
	2025-03-12 00:00:00: 7.02
	2025-03-14 00:00:00: 7.45
	2025-03-15 00:00:00: 8.05
	2025-03-16 00:00:00: 6.05
Anna Graf: 
	2025-03-16 00:00:00: 5.48
Ekaterina Grashchenko: 
	2025-03-11 00:00:00: 7.25
	2025-03-14 00:00:00: 7.05
	2025-03-16 00:00:00: 7.85
Laura Greidenweiß: 
	2025-03-10 00:00:00: 5.78
	2025-03-13 00:00:00: 7.37
	2025-03-15 00:00:00: 8.1
Svenja Halamek: 
	2

In [ ]:
class WorkHours:
    def __init__(self):
        return

class DailyWorkHours:
    def __init__(self, tip):
        self.tip = tip
        self.work_hours = {}

    def total_hours(self):
        return round(sum(self.work_hours.values()), 2)

    def __repr__(self):
        work_hours = '\n'.join([f'\t{name}: {duration}' for name, duration in self.work_hours.items()])
        total_hours = self.total_hours()
        tip_per_hour = round(self.tip / total_hours, 2)
        return f'total tip {self.tip}, total hours {total_hours}, tip per hour {tip_per_hour}\n{work_hours}'
    
class WeeklyTips:
    def __init__(self, daily_tips, employees_work_hours):
        days = sorted({work_day.day for employee in employees_work_hours for work_day in employee.work_days})
        self.weekly_work_times = {day: DailyWorkHours(tip) for day, tip in zip(days, daily_tips)}
        for employee in employees_work_hours:
            for work_day in employee.work_days:
                daily_work_hours = self.weekly_work_times[work_day.day]
                daily_work_hours.work_hours[employee.name] = work_day.hours

    def __repr__(self):
        return '\n'.join([f'{day.strftime("%d.%m.%Y")}: {daily_work_times}' for day, daily_work_times in self.weekly_work_times.items()])

daily_tips = [112.00, 177.71, 155.20, 96.22, 143.50, 181.50, 160.90]
weekly_work_times = WeeklyTips(daily_tips=daily_tips, employees_work_hours=employees_work_hours)
print(weekly_work_times)

10.03.2025: total tip 112.0, total hours 35.43, tip per hour 3.16
	Açelya Basili: 4.52
	Anna Sophie Blessing: 6.0
	Laura Greidenweiß: 5.78
	Mauro Sirigu: 11.7
	Jennifer Wöhrlin: 7.43
11.03.2025: total tip 177.71, total hours 60.27, tip per hour 2.95
	Emelie Appl: 5.05
	Manuel Bär: 7.98
	Markus Fuchs: 6.12
	Ekaterina Grashchenko: 7.25
	Philine Ostermayer: 8.38
	Damiano Parziale: 7.7
	Mauro Sirigu: 11.92
	Matthias Strobel: 5.87
12.03.2025: total tip 155.2, total hours 52.9, tip per hour 2.93
	Emelie Appl: 5.93
	Fabrizio Bellia: 11.0
	Anna Sophie Blessing: 5.82
	Sabulla Bullach: 7.13
	Markus Fuchs: 7.02
	Marie Jaussi: 8.32
	Luca Tegeder: 7.68
13.03.2025: total tip 96.22, total hours 47.99, tip per hour 2.01
	Fabrizio Bellia: 11.65
	Anna Sophie Blessing: 4.55
	Manuel Bär: 6.83
	Laura Greidenweiß: 7.37
	Svenja Halamek: 5.52
	Philine Ostermayer: 6.1
	Jennifer Wöhrlin: 5.97
14.03.2025: total tip 143.5, total hours 60.64, tip per hour 2.37
	Fabrizio Bellia: 6.07
	Sabulla Bullach: 6.97
	Manuel 